In [10]:
from collections import Counter
from sklearn.cluster import KMeans
from matplotlib import colors
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import ImageColor
from math import sqrt
import pandas as pd
import urllib

In [11]:
def rgb_to_hex(rgb_color):
    hex_color = "#"
    for i in rgb_color:
        i = int(i)
        hex_color += ("{:02x}".format(i))
    return hex_color

In [12]:
def prep_image(raw_img):
    modified_img = cv2.resize(raw_img, (900, 600), interpolation = cv2.INTER_AREA)
    modified_img = modified_img.reshape(modified_img.shape[0]*modified_img.shape[1], 3)
    return modified_img

In [25]:
def color_analysis(img):
    clf = KMeans(n_clusters = 2)
    color_labels = clf.fit_predict(img)
    center_colors = clf.cluster_centers_
    counts = Counter(color_labels)
    ordered_colors = [center_colors[i] for i in counts.keys()]
    hex_colors = [rgb_to_hex(ordered_colors[i]) for i in counts.keys()]
    plt.figure(figsize = (12, 8))
    #plt.pie(counts.values(), colors = hex_colors,autopct='%1.0f%%')
    print(counts.values())
    first_color=hex_colors[0]
    #plt.savefig("color_analysis_report.png")
    print(hex_colors[0])
    a=ImageColor.getcolor(first_color, "RGB")
    print(a)
    COLORS = {
                (0,0,0):"black",(31, 31, 32):"black",(119, 119, 121):"grey",(172, 162, 170):"grey",(255,255,255):"white",(245, 245, 220):"white",(255, 105, 180):"pink",
                (255,0,0):"red",(139,0,0):"red",(139,0,0):"red",(252,79,19):"orange",(205, 99, 85):"orange",
                (255,128,0):"orange",(254,101,33):"orange",(0,0,255):"blue",(255,255,0):"yellow",
                (249,217,94):"yellow",(255,166,0):"yellow",(193, 136, 53):"yellow",
                (0,128,0):"green",(128,0,128):"purple",(199, 143, 119):"brown",
                (255,165,0):"yellow",(255,215,0):"gold",(218,112,214):"purple",
                (255,20,147):"pink",(246, 195, 191):"cream",(139,69,19):"brown",(135,206,250):"blue",(1,119,193):"blue",(32, 42, 79):"blue",
                (39,1,188):"blue", (85,107,47):"green",(64, 109, 148):"blue",(34,35,55):"blackish blue"

                }

    def closest_color(rgb):
        r, g, b = rgb
        color_diffs = []
        for color in COLORS.keys():
            cr, cg, cb = color
            color_diff = sqrt(abs(r - cr)**2 + abs(g - cg)**2 + abs(b - cb)**2)
            color_diffs.append((color_diff, color))
            d=min(color_diffs)[1]
        return d

    print(COLORS[closest_color(a)])
    text = COLORS[closest_color(a)]
    csv1.loc[j, 'Color'] = f"{text}"
    csv1.to_csv(f'{sheet}', index = False)


In [5]:
def crop_img(img, scale=1.0):
    center_x, center_y = img.shape[1] / 2, img.shape[0] / 2
    width_scaled, height_scaled = img.shape[1] * scale, img.shape[0] * scale
    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
    img_cropped = img[int(top_y):int(bottom_y), int(left_x):int(right_x)]
    return img_cropped

In [26]:
sheet = "Results_Jan.csv"
csv1 = pd.read_csv(f'{sheet}')

for j in range (0,csv1.shape[0]):
    a = csv1.iloc[j][5]
    print(a)
    req = urllib.request.urlopen(f'{a}')
    arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
    img = cv2.imdecode(arr, -1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
            
    mask = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY)[1]

            # negate mask
    mask = 255 - mask

            # apply morphology to remove isolated extraneous noise
            # use borderconstant of black since foreground touches the edges
    kernel = np.ones((3,3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

            # anti-alias the mask -- blur then stretch
            # blur alpha channel
    mask = cv2.GaussianBlur(mask, (0,0), sigmaX=2, sigmaY=2, borderType = cv2.BORDER_DEFAULT)

            # linear stretch so that 127.5 goes to 0, but 255 stays 255
    mask = (2*(mask.astype(np.float32))-255.0).clip(0,255).astype(np.uint8)

            # put mask into alpha channel
    result = img.copy()
    result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
    result[:, :, 3] = mask
    result = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
    image = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
    
    modified_image = prep_image(image)
    color_analysis(modified_image)

https://media.6media.me/media/catalog/product/7/7/77717087321-1.jpg
dict_values([397004, 142996])
#dadada
(218, 218, 218)
white
https://media.6media.me/media/catalog/product/1/9/199657221-1.jpg
dict_values([437381, 102619])
#5e5e5e
(94, 94, 94)
grey
https://media.6media.me/media/catalog/product/2/4/244140856-1.jpg
dict_values([481167, 58833])
#ececec
(236, 236, 236)
white
https://media.6media.me/media/catalog/product/1/9/198844925-1.jpg
dict_values([486868, 53132])
#f2f2f2
(242, 242, 242)
white
https://media.6media.me/media/catalog/product/3/0/301348351-1.jpg
dict_values([413800, 126200])
#f6f6f6
(246, 246, 246)
white
https://media.6media.me/media/catalog/product/2/7/279654377-1.jpg
dict_values([431208, 108792])
#292929
(41, 41, 41)
black
https://media.6media.me/media/catalog/product/2/7/271525697-1.jpg
dict_values([413899, 126101])
#f6f6f6
(246, 246, 246)
white
https://media.6media.me/media/catalog/product/3/1/316751301-1.jpg
dict_values([319744, 220256])
#717171
(113, 113, 113)
grey


<ipython-input-25-94c67440e8bb>:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize = (12, 8))


ValueError: unknown url type: 'nan'

<Figure size 864x576 with 0 Axes>

<Figure size 864x576 with 0 Axes>

...

<Figure size 864x576 with 0 Axes>

<Figure size 864x576 with 0 Axes>

<Figure size 864x576 with 0 Axes>

<Figure size 864x576 with 0 Axes>

<Figure size 864x576 with 0 Axes>